In [1]:
import pandas as pd
import numpy as np

In [4]:
df_test = pd.read_parquet('test_with_feats_v2.parquet')
df_train = pd.read_parquet('train_with_feats_v2.parquet')

In [5]:
df_train

,buy_post,customer_id,receipt_sum_mean,receipt_sum_std,receipt_count,items_receipt_mean,items_receipt_mean_std,sum_delta_min,sum_delta_max,sum_delta_mean,...,second_half_max,second_half_min,second_half_std,diff,ratio,cosine_similarity,was_promo_mean,was_promo_std,was_1rub_mean,was_1rub_std
0,0,45006,522.613333,395.184669,3,6.333333,5.773503,-765.91,213.99,-275.960000,...,409.97,195.98,151.313780,-658.915000,0.314979,0.080332,0.666667,0.577350,0.000000,0.00000
1,0,45303,148.943478,88.418075,23,1.260870,0.751809,-250.00,288.99,-0.908182,...,259.99,39.99,80.267503,-2.173106,0.985520,0.086801,0.391304,0.499011,0.000000,0.00000
2,0,102488,186.978333,331.158229,6,3.000000,4.427189,-830.94,830.94,0.200000,...,860.93,29.99,473.797616,253.983333,5.233996,0.123457,0.833333,0.408248,0.000000,0.00000
3,0,119045,396.970000,191.625756,3,2.000000,1.000000,-300.97,-55.00,-177.985000,...,314.98,259.98,38.890873,-328.470000,0.466726,0.222222,0.666667,0.577350,0.333333,0.57735
4,0,157470,679.296667,621.355704,3,5.000000,3.605551,-806.96,-414.97,-610.965000,...,809.96,3.00,570.606888,-818.450000,0.331839,0.084444,1.000000,0.000000,0.333333,0.57735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,1,46598535,353.000000,129.722781,3,5.333333,5.859465,-242.00,40.00,-101.000000,...,299.00,259.00,28.284271,-222.000000,0.556886,0.125000,0.666667,0.577350,0.000000,0.00000
499996,1,46609455,528.290000,125.825938,3,4.333333,0.577350,-249.99,99.99,-75.000000,...,644.95,394.96,176.769624,-25.005000,0.954116,0.147929,0.000000,0.000000,0.000000,0.00000
499997,1,46617009,562.470000,321.818220,4,3.000000,1.414214,-584.98,130.01,-249.990000,...,699.97,114.99,413.643325,-309.980000,0.567948,0.166667,0.000000,0.000000,0.000000,0.00000
499998,1,46639170,363.306667,177.842347,3,3.666667,1.527525,119.99,229.98,174.985000,...,519.96,399.97,84.845743,289.975000,2.705836,0.355372,1.000000,0.000000,0.000000,0.00000


In [3]:

df_train_cut = df_train.iloc[:, :-11]

df_origin = pd.read_parquet('train_dataset')
df_customer_datediffpoststats = df_origin.groupby(['customer_id']).agg({'date_diff_post': ['mean', 'max', 'min']})
df_customer_datediffpoststats.reset_index(inplace=True)
df_customer_datediffpoststats.columns = ['customer_id', 'mean', 'max', 'min']
df_customer_lastdatediffpost = df_origin.sort_values(by=['startdatetime']).drop_duplicates('customer_id', keep='last')[['customer_id', 'date_diff_post']]
df_targets = pd.merge(df_customer_datediffpoststats, df_customer_lastdatediffpost, on='customer_id')

df_targets_nonchurn_only = df_targets.dropna()
df_with_feats = pd.merge(df_targets_nonchurn_only, df_train_cut, on='customer_id')

from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
df_feats = df_with_feats.iloc[:, 6:]


df_feats = scaler.fit_transform(df_feats)

In [2]:
df_train_v4 = pd.read_parquet('train_with_feats_v4.parquet')

In [3]:
df_train_v4

,buy_post,customer_id,receipt_sum_mean,receipt_sum_std,receipt_count,items_receipt_mean,items_receipt_mean_std,sum_delta_min,sum_delta_max,sum_delta_mean,...,is_sauce_mean,is_sauce_std,with_toilet_std,with_toilet_mean,food_court_std,food_court_mean,countable_std,countable_mean,external_area_std,external_area_mean
0,0,45006,522.613333,395.184669,3,6.333333,5.773503,-765.91,213.99,-275.960000,...,0.052632,0.229416,0.374634,0.157895,0.374634,0.842105,0.418854,0.210526,0.000000,0.000000
1,0,45303,148.943478,88.418075,23,1.260870,0.751809,-250.00,288.99,-0.908182,...,0.241379,0.435494,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0,102488,186.978333,331.158229,6,3.000000,4.427189,-830.94,830.94,0.200000,...,0.166667,0.383482,0.383482,0.833333,0.383482,0.166667,0.323381,0.111111,0.427793,0.777778
3,0,119045,396.970000,191.625756,3,2.000000,1.000000,-300.97,-55.00,-177.985000,...,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.516398,0.333333,0.000000,0.000000
4,0,157470,679.296667,621.355704,3,5.000000,3.605551,-806.96,-414.97,-610.965000,...,0.333333,0.487950,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,1,46598535,353.000000,129.722781,3,5.333333,5.859465,-242.00,40.00,-101.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.341565,0.125000,0.000000,1.000000
499996,1,46609455,528.290000,125.825938,3,4.333333,0.577350,-249.99,99.99,-75.000000,...,0.230769,0.438529,0.000000,1.000000,0.000000,0.000000,0.518875,0.461538,0.000000,1.000000
499997,1,46617009,562.470000,321.818220,4,3.000000,1.414214,-584.98,130.01,-249.990000,...,0.166667,0.389249,0.000000,1.000000,0.000000,0.000000,0.389249,0.166667,0.000000,0.000000
499998,1,46639170,363.306667,177.842347,3,3.666667,1.527525,119.99,229.98,174.985000,...,0.181818,0.404520,0.000000,0.000000,0.000000,1.000000,0.404520,0.181818,0.000000,0.000000


In [4]:
df_original_train = pd.read_parquet("train_dataset_hackaton2023_train.gzip")
df_targets_raw = df_original_train.sort_values(by=['startdatetime']).drop_duplicates('customer_id', keep='last')[['customer_id', 'date_diff_post']]

In [5]:
df_targets_raw.to_parquet('train_targets.parquet')

In [6]:
df_targets_raw

,customer_id,date_diff_post
15701653,24774791,NaN
5846902,9876699,NaN
21353897,34241371,NaN
21130854,33583919,NaN
4712985,8340331,NaN
...,...,...
24253993,46198579,44.0
23643988,41586357,23.0
13373504,22436078,5.0
20321652,30984546,3.0


In [7]:
df_train_feats = df_train_v4.drop(columns=['buy_post', 'customer_id'])

In [17]:
df_train_feats_non_churn = df_train_v4[df_train_v4.buy_post == 1].drop(columns=['buy_post', 'customer_id'])
df_targets_non_churn = df_targets_raw.dropna().date_diff_post

In [26]:
df_all = pd.merge(df_train_v4, df_targets_raw, on=['customer_id'])
df_all_dropna = df_all.dropna()
df_train_feats_non_churn = df_all_dropna.drop(columns=['buy_post', 'customer_id', 'date_diff_post'])
df_targets_non_churn = df_all_dropna.date_diff_post

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [28]:
pipeline = make_pipeline(StandardScaler(), LogisticRegression())
pipeline.fit(df_train_feats_non_churn, df_targets_non_churn)

C:\Users\rakhmerov\.virtualenvs\pythonProject3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
X_train_v4, X_val_v4, y_train_v4, y_val_v4 = train_test_split(df_train_feats_non_churn, df_targets_non_churn, test_size=0.2,
                                                  random_state=42)


np.sqrt(mean_squared_error(y_))

In [29]:
df_5 = pd.read_parquet('train_with_feats_v5.parquet')

In [30]:
df_5

,buy_post,customer_id,receipt_sum_mean,receipt_sum_std,receipt_count,items_receipt_mean,items_receipt_mean_std,sum_delta_min,sum_delta_max,sum_delta_mean,...,countable_mean,external_area_std,external_area_mean,Friday_perc,Monday_perc,Saturday_perc,Sunday_perc,Thursday_perc,Tuesday_perc,Wednesday_perc
0,0,45006,522.613333,395.184669,3,6.333333,5.773503,-765.91,213.99,-275.960000,...,0.210526,0.000000,0.000000,0.00,0.00,0.00,0.00,0.67,0.00,0.33
1,0,45303,148.943478,88.418075,23,1.260870,0.751809,-250.00,288.99,-0.908182,...,0.000000,0.000000,0.000000,0.13,0.26,0.09,0.09,0.13,0.13,0.17
2,0,102488,186.978333,331.158229,6,3.000000,4.427189,-830.94,830.94,0.200000,...,0.111111,0.427793,0.777778,0.33,0.00,0.33,0.00,0.00,0.17,0.17
3,0,119045,396.970000,191.625756,3,2.000000,1.000000,-300.97,-55.00,-177.985000,...,0.333333,0.000000,0.000000,0.00,0.00,0.00,1.00,0.00,0.00,0.00
4,0,157470,679.296667,621.355704,3,5.000000,3.605551,-806.96,-414.97,-610.965000,...,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,1,46598535,353.000000,129.722781,3,5.333333,5.859465,-242.00,40.00,-101.000000,...,0.125000,0.000000,1.000000,0.00,0.67,0.00,0.33,0.00,0.00,0.00
499996,1,46609455,528.290000,125.825938,3,4.333333,0.577350,-249.99,99.99,-75.000000,...,0.461538,0.000000,1.000000,0.00,0.33,0.00,0.33,0.00,0.33,0.00
499997,1,46617009,562.470000,321.818220,4,3.000000,1.414214,-584.98,130.01,-249.990000,...,0.166667,0.000000,0.000000,0.00,1.00,0.00,0.00,0.00,0.00,0.00
499998,1,46639170,363.306667,177.842347,3,3.666667,1.527525,119.99,229.98,174.985000,...,0.181818,0.000000,0.000000,0.00,1.00,0.00,0.00,0.00,0.00,0.00
